# Family Graph Notebook (Interactive)

This notebook loads a JSON list of combined houses, converts each record into a `FamilyMember` instance using Pydantic, creates an undirected graph based on the parents and children relationships, and then displays an interactive graph using Pyvis. Node labels use the English name (`name.english`) if available, falling back to `name.hanzi`.

In [6]:
import json
from pathlib import Path

import networkx as nx

from src.models import FamilyMember


def load_family_members(json_path: Path):
    """
    Load the JSON list file and convert each record into a FamilyMember instance.
    """
    with open(json_path, "r", encoding="utf-8") as infile:
        data = json.load(infile)

    members = []
    for record in data:
        try:
            member = FamilyMember.parse_obj(record)
            members.append(member)
        except Exception as e:
            print(f"Error parsing record: {record}\n{e}")
    return members


def get_member_key(member: FamilyMember) -> str:
    """
    Get a unique key for the member.
    Prefer name.english; if missing, use name.hanzi.
    """
    return member.name.english if member.name.english else member.name.hanzi


def create_family_graph(members):
    """
    Create an undirected graph from a list of FamilyMember instances.
    Nodes are keyed by the canonical family member name (English name if available).
    Edges are added for both 'parents' and 'children' relationships.
    """
    G = nx.Graph()
    # Build a mapping from key -> member instance
    mapping = {}
    for member in members:
        node_label = get_member_key(member)  # English name if available
        mapping[node_label] = member
        # Add node with the label set to the English name
        G.add_node(node_label, label=node_label, data=member.dict())

    # For each member, connect to parents and children (undirected)
    for member in members:
        child_key = get_member_key(member)
        # Process parent's relationships
        for parent in member.parents:
            if isinstance(parent, str):
                parent_key = parent
            else:
                parent_key = get_member_key(parent)
            # Add edge if both nodes exist and avoid self-loop
            if parent_key in G.nodes and parent_key != child_key:
                G.add_edge(child_key, parent_key)
        # Process children relationships
        for child in member.children:
            if isinstance(child, str):
                ch_key = child
            else:
                ch_key = get_member_key(child)
            if ch_key in G.nodes and ch_key != child_key:
                G.add_edge(child_key, ch_key)
    return G


In [ ]:
# Set path for the combined JSON file
json_path = Path("data/combined_houses.json")

# Load family members
members = load_family_members(json_path)
print(f"Loaded {len(members)} family members.")

# Create family graph
family_graph = create_family_graph(members)
print(f"Graph has {family_graph.number_of_nodes()} nodes and {family_graph.number_of_edges()} edges.")

# Use Pyvis for an interactive plot
from pyvis.network import Network
net = Network(notebook=True, height="700px", width="100%", directed=False)

# Load the NetworkX graph into Pyvis
net.from_nx(family_graph)

# Set some basic options (customize as needed)
net.set_options("""
{
  "nodes": {
    "font": {
      "size": 16,
      "face": "arial"
    }
  },
  "physics": {
    "barnesHut": {
      "gravitationalConstant": -8000,
      "centralGravity": 0.3,
      "springLength": 95
    }
  }
}
"""
)

# Save and display the interactive graph
net.show("family_graph_interactive.html")

# (Optional) Embed the interactive graph in the notebook using an IFrame
from IPython.display import IFrame
IFrame(src="family_graph_interactive.html", width="100%", height="700px", cdn_resources='inlined')


Loaded 80 family members.
Graph has 79 nodes and 95 edges.
family_graph_interactive.html


/tmp/ipykernel_21955/2924579710.py:19: PydanticDeprecatedSince20: The `parse_obj` method is deprecated; use `model_validate` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  member = FamilyMember.parse_obj(record)
/tmp/ipykernel_21955/2924579710.py:47: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  G.add_node(node_label, label=node_label, data=member.dict())


The above cell loads the family members, creates the family graph, and then uses Pyvis to display an interactive network. The nodes display the English name (from `name.english`, falling back to `name.hanzi`).